In [ ]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render.rendered_html{font-size:15pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:18pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:18pt;padding:5px;}
table.dataframe{font-size:18px;}
</style>
"""))

In [15]:
import json
import pandas as pd
from IPython.display import JSON, display

# JSON 파일 읽기
with open('신림동_짜장면/images_info.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 표 형태로 보기
df = pd.DataFrame(data['이미지_목록'])
display(df.head())

,번호,파일이름,url,rendered_size,intrinsic_size,intrinsic_aspect_ratio,이미지제목,source,등록일자,is_base64,actual_file_size,EXIF_날짜,estimated_file_size
0,2,2.jpg,https://encrypted-tbn0.gstatic.com/images?q=tb...,285 × 162 px,311 × 162 px,311∶162,"그 시절 먹었던 추억의 그 맛…"" 긴 세월 자리를 지켜온 신림 맛집 TOP 3",Unknown,Unknown,False,10.6 KB,None,NaN
1,5,5.jpg,https://encrypted-tbn0.gstatic.com/images?q=tb...,160 × 180 px,194 × 259 px,194∶259,신림] 어마어마한 양에 가성비 좋은 중국집 추천 / 선짜장 : 네이버 블로그,Unknown,Unknown,False,12.1 KB,None,NaN
2,6,6.jpg,https://encrypted-tbn0.gstatic.com/images?q=tb...,250 × 180 px,275 × 183 px,275∶183,황금짜장 | 완벽한 하루,Unknown,Unknown,False,11.4 KB,None,NaN
3,7,7.jpg,https://encrypted-tbn0.gstatic.com/images?q=tb...,207 × 180 px,224 × 225 px,224∶225,강감찬관악종합사회복지관 - 행복과 사랑이 가득했던 관중회와 함께한 '사랑의 짜장면',Unknown,Unknown,False,11.5 KB,None,NaN
4,8,8.jpg,https://encrypted-tbn0.gstatic.com/images?q=tb...,263 × 168 px,299 × 168 px,299∶168,"짜장면 값이 올라서 3,000원??? 신림의 33년 전통 중식당 강남반점입니다. -...",Unknown,Unknown,False,17.4 KB,None,NaN


In [25]:
### 왜 안되냐,,,, 다시 url 정리

In [37]:
# ====================
# 🔧 설정 부분 - 여기만 수정하세요!
# ====================
SEARCH_QUERY = "짜장면"  # 🔍 고급 검색어 (제외 키워드 포함)
MAX_DOWNLOAD = 1000           # 📥 다운로드할 최대 이미지 수
MAX_SCROLLS = 15            # 🔄 최대 스크롤 횟수 
SCROLL_PAUSE_TIME = 1       # ⏱️ 스크롤 간 대기 시간(초)

# 🎯 이미지 필터링 설정
FILTER_SETTINGS = {
    "use_filters": True,                    # 필터링 사용 여부
    "exclude_keywords": [                   # 제외할 키워드들 (이미지 제목에서)
        "짬뽕", "간판", "메뉴", "가게", "식당", "매장", 
        "배달", "포장", "주문", "리뷰", "맛집", "음식점",
        "간판", "로고", "메뉴판", "price", "menu", "restaurant"
    ],
    "required_keywords": [                  # 포함되어야 하는 키워드 (하나라도 있으면 됨)
        "짜장면", "자장면", "볶음짜장", "간짜장", "jajangmyeon"
    ],
    "min_image_size": {"width": 200, "height": 150},  # 최소 이미지 크기
    "exclude_small_files": True,            # 5KB 미만 파일 제외
    "exclude_duplicates": True              # 비슷한 이미지 제외 (URL 기준)
}
# ====================

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import urllib.request
import json
import os
from PIL import Image
import base64
import io

def create_folder(search_query):
    """검색어로 폴더 생성"""
    folder_name = search_query.replace(" ", "_").replace("-", "").replace('"', '')
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    return folder_name

def save_combined_info(folder_path, all_images_info, search_query):
    """모든 이미지 정보를 하나의 JSON 파일로 저장 (상세 정보 포함)"""
    # 각 이미지 정보에 상세 메타데이터 추가
    for image_info in all_images_info:
        # 파일 정보 추가
        image_info["파일정보"] = {
            "파일명": image_info.get('파일이름', 'Unknown'),
            "파일크기": image_info.get('actual_file_size', 'Unknown'),
            "생성일시": time.strftime('%Y-%m-%d %H:%M:%S'),
            "크롤링_소스": "Google Images",
            "다운로드_방법": image_info.get('download_method', 'Unknown')
        }
        
        # 검색 정보 추가
        image_info["검색정보"] = {
            "검색어": search_query,
            "검색일시": time.strftime('%Y-%m-%d %H:%M:%S'),
            "이미지_번호": image_info.get('번호', 'Unknown'),
            "필터_설정": FILTER_SETTINGS if FILTER_SETTINGS["use_filters"] else "필터 사용 안함"
        }
        
        # EXIF 정보가 있으면 추가
        if image_info.get('EXIF_날짜') and image_info.get('EXIF_날짜') != 'None':
            image_info["메타데이터"] = {
                "EXIF_촬영날짜": image_info.get('EXIF_날짜', 'None'),
                "원본_EXIF_존재": "Yes" if image_info.get('EXIF_날짜') != 'None' else "No"
            }
    
    combined_info = {
        "크롤링_요약": {
            "검색어": search_query,
            "총_이미지_수": len(all_images_info),
            "생성일시": time.strftime("%Y-%m-%d %H:%M:%S"),
            "크롤링_설정": {
                "최대_다운로드": MAX_DOWNLOAD,
                "최대_스크롤": MAX_SCROLLS,
                "필터_사용": FILTER_SETTINGS["use_filters"],
                "제외_키워드": FILTER_SETTINGS["exclude_keywords"] if FILTER_SETTINGS["use_filters"] else [],
                "필수_키워드": FILTER_SETTINGS["required_keywords"] if FILTER_SETTINGS["use_filters"] else []
            }
        },
        "이미지_목록": all_images_info
    }
    
    combined_file = os.path.join(folder_path, "images_info.json")
    with open(combined_file, 'w', encoding='utf-8') as f:
        json.dump(combined_info, f, ensure_ascii=False, indent=2)
    
    print(f"📄 통합 메타데이터 파일 저장: {combined_file}")
    print(f"📋 포함된 정보: 이미지 {len(all_images_info)}개의 모든 메타데이터")

def apply_google_search_filters(driver):
    """구글 이미지 검색에서 필터 적용"""
    try:
        print("🎯 구글 검색 필터 적용 중...")
        
        # 도구 버튼 클릭
        try:
            tools_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//div[text()='도구' or text()='Tools']"))
            )
            tools_button.click()
            time.sleep(1)
        except:
            print("도구 버튼을 찾을 수 없습니다.")
            return
        
        # 크기 필터: 중간 크기 이상
        try:
            size_button = driver.find_element(By.XPATH, "//div[text()='크기' or text()='Size']")
            size_button.click()
            time.sleep(0.5)
            
            medium_size = driver.find_element(By.XPATH, "//div[text()='중간' or text()='Medium']")
            medium_size.click()
            time.sleep(1)
            print("✅ 크기 필터: 중간 이상 적용")
        except:
            print("❌ 크기 필터 적용 실패")
        
        # 유형 필터: 사진만
        try:
            type_button = driver.find_element(By.XPATH, "//div[text()='유형' or text()='Type']")
            type_button.click()
            time.sleep(0.5)
            
            photo_type = driver.find_element(By.XPATH, "//div[text()='사진' or text()='Photos']")
            photo_type.click()
            time.sleep(1)
            print("✅ 유형 필터: 사진만 적용")
        except:
            print("❌ 유형 필터 적용 실패")
            
        time.sleep(2)  # 필터 적용 대기
        
    except Exception as e:
        print(f"필터 적용 중 오류: {e}")

def is_image_relevant(image_info):
    """이미지가 관련성이 있는지 확인"""
    if not FILTER_SETTINGS["use_filters"]:
        return True
    
    title = image_info.get("이미지제목", "").lower()
    url = image_info.get("url", "").lower()
    source = image_info.get("source", "").lower()
    
    combined_text = f"{title} {url} {source}"
    
    # 1. 제외 키워드 확인
    for exclude_word in FILTER_SETTINGS["exclude_keywords"]:
        if exclude_word.lower() in combined_text:
            print(f"❌ 제외됨 (키워드: {exclude_word}): {title[:30]}...")
            return False
    
    # 2. 필수 키워드 확인 (하나라도 있으면 통과)
    if FILTER_SETTINGS["required_keywords"]:
        has_required = False
        for required_word in FILTER_SETTINGS["required_keywords"]:
            if required_word.lower() in combined_text:
                has_required = True
                break
        
        if not has_required:
            print(f"❌ 제외됨 (필수 키워드 없음): {title[:30]}...")
            return False
    
    # 3. 파일 크기 확인
    if FILTER_SETTINGS["exclude_small_files"]:
        try:
            file_size_str = image_info.get("actual_file_size", "")
            if file_size_str:
                size_kb = float(file_size_str.split()[0])
                if size_kb < 5:  # 5KB 미만
                    print(f"❌ 제외됨 (파일 작음: {size_kb}KB): {title[:30]}...")
                    return False
        except:
            pass
    
    return True

def remove_duplicate_images(all_images_info):
    """중복 이미지 제거"""
    if not FILTER_SETTINGS["exclude_duplicates"]:
        return all_images_info
    
    seen_urls = set()
    unique_images = []
    
    for img_info in all_images_info:
        url = img_info.get("url", "")
        # URL에서 쿼리 파라미터 제거하여 비교
        clean_url = url.split('?')[0] if url else ""
        
        if clean_url not in seen_urls and clean_url:
            seen_urls.add(clean_url)
            unique_images.append(img_info)
        else:
            print(f"❌ 중복 제거: {img_info.get('이미지제목', 'Unknown')[:30]}...")
    
    removed_count = len(all_images_info) - len(unique_images)
    if removed_count > 0:
        print(f"🗑️ 중복 이미지 {removed_count}개 제거됨")
    
    return unique_images

def download_image(url, file_path):
    """이미지 다운로드 (기본 방법 - URL 유효성 검사 강화)"""
    try:
        # URL 유효성 검사
        if not url or not isinstance(url, str):
            print(f"❌ 유효하지 않은 URL 타입: {type(url)}")
            return False
            
        if len(url) < 10:
            print(f"❌ URL이 너무 짧음: {repr(url)}")
            return False
            
        if not url.startswith(("http://", "https://", "data:image")):
            print(f"❌ 지원하지 않는 URL 프로토콜: {url[:50]}...")
            return False
        
        print(f"🔗 기본 다운로드 시도: {url[:80]}...")
        
        # data:image URL 처리
        if url.startswith("data:image"):
            try:
                if "," in url:
                    header, data = url.split(",", 1)
                    image_data = base64.b64decode(data)
                    
                    with open(file_path, 'wb') as f:
                        f.write(image_data)
                    
                    print(f"✅ Base64 이미지 저장 성공")
                    return True
                else:
                    print(f"❌ 잘못된 Base64 형식")
                    return False
            except Exception as e:
                print(f"❌ Base64 처리 실패: {e}")
                return False
        
        # HTTP/HTTPS URL 처리
        else:
            opener = urllib.request.build_opener()
            opener.addheaders = [
                ('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'),
                ('Referer', 'https://www.google.com/'),
                ('Accept', 'image/*,*/*;q=0.8')
            ]
            urllib.request.install_opener(opener)
            
            # 타임아웃 설정하여 다운로드
            urllib.request.urlretrieve(url, file_path)
            
            # 파일 크기 확인
            if os.path.exists(file_path):
                file_size = os.path.getsize(file_path)
                if file_size > 0:
                    print(f"✅ HTTP 다운로드 성공: {file_size / 1024:.1f}KB")
                    return True
                else:
                    print(f"❌ 빈 파일 다운로드됨")
                    os.remove(file_path)
                    return False
            else:
                print(f"❌ 파일이 생성되지 않음")
                return False
        
    except Exception as e:
        print(f"❌ 기본 다운로드 실패: {e}")
        # 실패한 파일이 있다면 삭제
        if os.path.exists(file_path):
            try:
                os.remove(file_path)
            except:
                pass
        return False

def get_image_metadata_date(file_path):
    """다운로드된 이미지 파일에서 EXIF 날짜 정보 추출"""
    try:
        from PIL import Image
        from PIL.ExifTags import TAGS
        
        image = Image.open(file_path)
        exifdata = image.getexif()
        
        date_fields = ['DateTime', 'DateTimeOriginal', 'DateTimeDigitized']
        
        for tag_id in exifdata:
            tag = TAGS.get(tag_id, tag_id)
            if tag in date_fields:
                return str(exifdata.get(tag_id))
        
        return None
    except:
        return None

# 개별 메타데이터 파일 생성 함수들 제거 - 통합 JSON만 사용

def get_image_info(driver, image_element, count):
    """이미지 정보 추출"""
    try:
        print(f"🔍 이미지 {count} 정보 추출 중...")
        
        # 먼저 기본 이미지 URL 확인
        basic_src = image_element.get_attribute("src") or image_element.get_attribute("data-src")
        
        # 이미지 정보 수집
        image_info = {}
        image_info["번호"] = count
        image_info["파일이름"] = f"{count}.jpg"
        
        # 기본 URL이 유효한지 확인
        if basic_src and isinstance(basic_src, str) and len(basic_src) > 20:
            if basic_src.startswith(("http://", "https://", "data:image")):
                print(f"  ✅ 기본 URL 발견: {basic_src[:80]}...")
                original_urls = [basic_src]
            else:
                print(f"  ❌ 유효하지 않은 기본 URL: {repr(basic_src)}")
                original_urls = []
        else:
            print(f"  ⚠️ 기본 URL 없음 또는 너무 짧음: {repr(basic_src)}")
            original_urls = []
        
        # 기본 URL이 없으면 클릭해서 더 큰 이미지 찾기
        if not original_urls:
            print(f"  🔍 클릭하여 큰 이미지 탐색...")
            try:
                # 이미지 클릭
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", image_element)
                time.sleep(0.3)
                driver.execute_script("arguments[0].click();", image_element)
                time.sleep(1.5)
                
                # 큰 이미지 찾기
                large_img_selectors = [
                    "//img[@class='sFlh5c pT0Scc iPVvYb']",
                    "//img[@class='n3VNCb KAlRDb']", 
                    "//img[@class='iPVvYb']",
                    "//img[contains(@class, 'sFlh5c')]"
                ]
                
                for selector in large_img_selectors:
                    try:
                        large_imgs = driver.find_elements(By.XPATH, selector)
                        for img in large_imgs:
                            src = img.get_attribute("src")
                            if src and isinstance(src, str) and len(src) > 20:
                                if src.startswith(("http://", "https://", "data:image")):
                                    if src not in original_urls:
                                        original_urls.append(src)
                                        print(f"  ✅ 큰 이미지 URL 발견: {src[:80]}...")
                    except:
                        continue
                        
            except Exception as e:
                print(f"  ❌ 클릭 탐색 실패: {e}")
        
        # 여전히 URL이 없으면 스킵
        if not original_urls:
            print(f"  ❌ 유효한 URL을 찾을 수 없습니다.")
            return None
        
        image_info["url"] = original_urls[0]
        
        # 이미지 기본 정보
        try:
            width = image_element.get_attribute("width") or image_element.size.get('width', 0)
            height = image_element.get_attribute("height") or image_element.size.get('height', 0)
            image_info["preview_size"] = f"{width} × {height} px"
        except:
            image_info["preview_size"] = "Unknown"
        
        # 이미지 제목
        try:
            title = image_element.get_attribute("alt") or image_element.get_attribute("title") or "Unknown"
            image_info["이미지제목"] = title.strip() if title and title.strip() else "Unknown"
        except:
            image_info["이미지제목"] = "Unknown"
        
        # 기본 정보들
        image_info["source"] = "Unknown"
        image_info["등록일자"] = "Unknown"
        image_info["is_base64"] = original_urls[0].startswith("data:image") if original_urls else False
        
        print(f"  ✅ 이미지 {count} 정보 추출 완료")
        return image_info, original_urls
        
    except Exception as e:
        print(f"  ❌ 이미지 정보 추출 실패 (이미지 {count}): {e}")
        return None

def main():
    # 설정된 검색어 사용
    search_query = SEARCH_QUERY
    print(f"🔍 검색어: '{search_query}'")
    print(f"📥 최대 다운로드: {MAX_DOWNLOAD}개")
    print(f"🔄 최대 스크롤: {MAX_SCROLLS}회")
    print("-" * 50)
    
    # 폴더 생성
    folder_path = create_folder(search_query)
    
    # 웹드라이버 설정
    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    
    driver = webdriver.Chrome(options=options)
    
    try:
        # 구글 이미지 검색
        driver.get("https://www.google.co.kr/imghp?hl=ko&tab=wi&authuser=0&ogbl")
        
        # 검색어 입력
        search_box = driver.find_element(By.NAME, "q")
        search_box.send_keys(search_query)
        search_box.send_keys(Keys.RETURN)
        
        # 페이지 로딩 대기
        time.sleep(3)
        
        # 구글 검색 필터 적용
        apply_google_search_filters(driver)
        
        # 스크롤하여 더 많은 이미지 로드 (스마트 종료)
        last_height = driver.execute_script("return document.body.scrollHeight")
        scroll_count = 0
        more_button_clicked = False
        no_change_count = 0  # 변화 없는 횟수 카운터
        max_no_change = 3    # 3번 연속 변화 없으면 종료
        last_image_count = 0  # 이전 이미지 수
        
        print("이미지 로딩 중...")
        print(f"🔄 최대 스크롤: {MAX_SCROLLS}회 (변화 없으면 자동 종료)")
        
        while scroll_count < MAX_SCROLLS:
            print(f"📜 스크롤 {scroll_count + 1}/{MAX_SCROLLS} 진행 중...")
            
            # 스크롤 다운
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            
            # 새로운 높이 계산
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            # 현재 로드된 이미지 수 확인
            current_images = driver.find_elements(By.CSS_SELECTOR, ".mNsIhb, .YQ4gaf, .rg_i, img")
            current_image_count = len(current_images)
            
            print(f"   📊 현재 이미지 수: {current_image_count}개 (높이: {new_height})")
            
            # 높이나 이미지 수에 변화가 있는지 확인
            height_changed = new_height != last_height
            images_increased = current_image_count > last_image_count
            
            if height_changed or images_increased:
                # 변화가 있으면 카운터 리셋
                no_change_count = 0
                last_image_count = current_image_count
                print(f"   ✅ 변화 감지됨")
            else:
                # 변화가 없으면 카운터 증가
                no_change_count += 1
                print(f"   ⚠️ 변화 없음 ({no_change_count}/{max_no_change})")
                
                if no_change_count >= max_no_change:
                    print(f"   🛑 {max_no_change}번 연속 변화 없음 - 더보기 버튼 확인 중...")
                    
                    # "결과 더보기" 버튼 클릭 시도
                    more_button_selectors = [
                        ".GNJvt.ipz2Oe",
                        ".mye4qd",
                        "[jsname='Lo5Rub']",
                        "input[value='결과 더보기']",
                        "input[value='Show more results']"
                    ]
                    
                    button_found = False
                    for selector in more_button_selectors:
                        try:
                            buttons = driver.find_elements(By.CSS_SELECTOR, selector)
                            for button in buttons:
                                try:
                                    if button.is_displayed() and button.is_enabled():
                                        print(f"   🔘 더보기 버튼 발견!")
                                        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
                                        time.sleep(1)
                                        driver.execute_script("arguments[0].click();", button)
                                        more_button_clicked = True
                                        button_found = True
                                        print(f"   ✅ 더보기 버튼 클릭 성공!")
                                        time.sleep(3)
                                        no_change_count = 0  # 카운터 리셋
                                        break
                                except Exception as e:
                                    print(f"   ❌ 버튼 클릭 실패: {e}")
                                    continue
                            if button_found:
                                break
                        except Exception as e:
                            continue
                    
                    if not button_found:
                        print(f"   🚫 더보기 버튼을 찾을 수 없습니다.")
                        print(f"   🏁 더 이상 로드할 이미지가 없는 것 같습니다. 스크롤 종료!")
                        break
            
            last_height = new_height
            scroll_count += 1
        
        # 스크롤 완료 메시지
        if scroll_count >= MAX_SCROLLS:
            print(f"🔄 최대 스크롤 횟수({MAX_SCROLLS})에 도달했습니다.")
        else:
            print(f"✅ 스크롤 자동 종료 (총 {scroll_count}회 스크롤)")
        
        if more_button_clicked:
            print("🔘 결과 더보기 버튼을 클릭했습니다!")
        
        # 최종 이미지 수 확인
        final_images = driver.find_elements(By.CSS_SELECTOR, ".mNsIhb, .YQ4gaf, .rg_i, img")
        print(f"📊 스크롤 완료 후 총 로드된 요소: {len(final_images)}개")
        
        # 이미지 요소들 찾기
        print("이미지 요소 탐지 중...")
        
        # 모든 가능한 이미지 요소들을 찾기 위한 다양한 선택자
        google_image_selectors = [
            ".mNsIhb",                # 최신 구조
            ".YQ4gaf",                # 최신 구조
            ".czzyk.XOEbc",           # 다른 구조
            ".rg_i",                  # 기존 구조
            "img[src*='googleusercontent']",  # 구글 이미지 서버
            "img[src*='gstatic']",    # 구글 정적 이미지
        ]
        
        found_images = set()  # 중복 제거를 위한 set
        
        for selector in google_image_selectors:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    # img 태그가 아닌 경우 내부에서 img 찾기
                    if element.tag_name != 'img':
                        inner_imgs = element.find_elements(By.TAG_NAME, "img")
                        for img in inner_imgs:
                            found_images.add(img)
                    else:
                        found_images.add(element)
                print(f"'{selector}' 선택자로 {len(elements)}개 요소 발견")
            except Exception as e:
                print(f"'{selector}' 선택자 오류: {e}")
                continue
        
        # 실제 이미지 URL이 있는 것들만 필터링
        valid_images = []
        for img in found_images:
            try:
                src = img.get_attribute("src") or img.get_attribute("data-src")
                if src and (
                    src.startswith("http") or 
                    src.startswith("data:image") or
                    "googleusercontent" in src or
                    "gstatic" in src
                ):
                    # 너무 작은 이미지나 아이콘 제외
                    try:
                        width = int(img.get_attribute("width") or 0)
                        height = int(img.get_attribute("height") or 0)
                        if width < 50 or height < 50:  # 50px 미만은 제외
                            continue
                    except:
                        pass
                    
                    valid_images.append(img)
            except:
                continue
        
        images = valid_images
        print(f"🎯 유효한 이미지 수: {len(images)}개")
        
        if not images:
            print("❌ 유효한 이미지를 찾을 수 없습니다.")
            return
        
        total_images = len(images)
        print(f"🔍 총 검색된 이미지 수: {total_images}개")
        
        # 다운로드할 이미지 수 설정
        max_download = min(MAX_DOWNLOAD, total_images)
        print(f"📥 다운로드 예정 이미지 수: {max_download}개")
        
        if total_images > MAX_DOWNLOAD:
            print(f"⚠️  총 {total_images}개 중 {MAX_DOWNLOAD}개만 다운로드합니다.")
        
        count = 1
        success_count = 0
        failed_count = 0
        filtered_count = 0
        all_images_info = []  # 모든 이미지 정보를 저장할 리스트
        
        for image in images[:max_download]:
            try:
                print(f"📸 이미지 {count}/{max_download} 처리 중...")
                
                # 이미지 정보 추출
                result = get_image_info(driver, image, count)
                if result is None:
                    print(f"❌ 이미지 {count} 정보 추출 실패")
                    failed_count += 1
                    count += 1
                    continue
                
                image_info, original_urls = result
                
                # 이미지 저장
                if original_urls:
                    file_path = os.path.join(folder_path, f"{count}.jpg")
                    
                    print(f"📸 이미지 {count} 다운로드 시작")
                    
                    # 첫 번째 URL로 다운로드 시도
                    first_url = original_urls[0]
                    download_success = download_image(first_url, file_path)
                    
                    if download_success:
                        print(f"✅ 이미지 {count} 다운로드 성공!")
                        
                        # 파일 크기 정보 업데이트
                        file_size = os.path.getsize(file_path) / 1024
                        image_info["actual_file_size"] = f"{file_size:.1f} KB"
                        image_info["download_method"] = "Basic download"
                        
                        # EXIF 날짜 정보 확인
                        exif_date = get_image_metadata_date(file_path)
                        if exif_date:
                            image_info["EXIF_날짜"] = exif_date
                        else:
                            image_info["EXIF_날짜"] = "None"
                        
                        # 검색어 정보 추가
                        image_info["검색어"] = search_query
                        
                        # 관련성 확인 (필터링)
                        if is_image_relevant(image_info):
                            print(f"✅ 이미지 {count} 저장 완료 ({file_size:.1f} KB) - 관련성 ✓")
                            success_count += 1
                            all_images_info.append(image_info)
                        else:
                            # 관련성이 없으면 파일 삭제
                            try:
                                os.remove(file_path)
                                print(f"🗑️ 관련성 없는 이미지 삭제: {count}")
                            except:
                                pass
                            filtered_count += 1
                    else:
                        print(f"❌ 이미지 {count} 다운로드 실패")
                        failed_count += 1
                else:
                    print(f"❌ 이미지 {count} URL을 찾을 수 없음")
                    failed_count += 1
                
                count += 1
                
                # 진행률 표시
                progress = (count - 1) / max_download * 100
                print(f"📊 진행률: {progress:.1f}% (성공: {success_count}, 실패: {failed_count}, 필터됨: {filtered_count})")
                
            except Exception as e:
                print(f"❌ 이미지 {count} 처리 중 오류: {e}")
                failed_count += 1
                count += 1
                continue
        
        # 중복 이미지 제거
        if all_images_info:
            all_images_info = remove_duplicate_images(all_images_info)
        
        # 통합 JSON 파일 생성
        if all_images_info:
            save_combined_info(folder_path, all_images_info, search_query)
        
        print(f"\n🎉 크롤링 완료!")
        print(f"📊 최종 결과:")
        print(f"   🔍 검색어: '{search_query}'")
        print(f"   🖼️  총 검색된 이미지: {total_images}개")
        print(f"   📥 다운로드 시도: {max_download}개")
        print(f"   ✅ 성공: {success_count}개")
        print(f"   ❌ 실패: {failed_count}개")
        print(f"   🗑️ 필터링됨: {filtered_count}개")
        print(f"   📂 저장 폴더: '{folder_path}'")
        
        if max_download > 0:
            print(f"   📈 성공률: {success_count/max_download*100:.1f}%")
            # 0으로 나누기 오류 방지
            if (success_count + filtered_count) > 0:
                print(f"   🎯 관련성 비율: {success_count/(success_count+filtered_count)*100:.1f}%")
            else:
                print(f"   🎯 관련성 비율: N/A (처리된 이미지 없음)")
        else:
            print(f"   📈 성공률: N/A (시도된 이미지 없음)")
        
        # 저장된 파일 목록 확인
        saved_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
        info_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
        
        print(f"\n📁 저장된 파일:")
        print(f"   🖼️  이미지 파일: {len(saved_files)}개")
        print(f"   📋 통합 메타데이터 파일: {'images_info.json' if 'images_info.json' in info_files else '없음'}")
        
        if success_count > 0:
            print(f"\n🎯 사용법:")
            print(f"   📂 이미지 확인: '{folder_path}' 폴더 열기")
            print(f"   📄 메타데이터 확인: 'images_info.json' 파일 열기")
            print(f"   💻 주피터에서 분석:")
            print(f"      import json")
            print(f"      with open('{folder_path}/images_info.json', 'r', encoding='utf-8') as f:")
            print(f"          data = json.load(f)")
            print(f"      print(f\"검색어: {{data['크롤링_요약']['검색어']}}\")")
            print(f"      images = data['이미지_목록']")
            print(f"   🔍 필터링 설정 조정: 코드 상단 FILTER_SETTINGS 수정")
        
        return {
            'search_query': search_query,
            'total_found': total_images,
            'attempted': max_download,
            'success': success_count,
            'failed': failed_count,
            'filtered': filtered_count,
            'folder_path': folder_path
        }
        
    finally:
        pass
#         driver.quit()

if __name__ == "__main__":
    # 여러 검색어로 연속 실행하고 싶다면 아래처럼 사용하세요:
    # search_queries = ["짜장면", "볶음밥", "탕수육"]
    # for query in search_queries:
    #     SEARCH_QUERY = query
    #     print(f"\n{'='*60}")
    #     print(f"🚀 '{query}' 검색 시작!")
    #     print(f"{'='*60}")
    #     result = main()
    #     print(f"✨ '{query}' 검색 완료: {result['success']}개 성공")
    
    # 단일 검색 실행
    main()

🔍 검색어: '짜장면'
📥 최대 다운로드: 1000개
🔄 최대 스크롤: 15회
--------------------------------------------------
🎯 구글 검색 필터 적용 중...
❌ 크기 필터 적용 실패
❌ 유형 필터 적용 실패
이미지 로딩 중...
🔄 최대 스크롤: 15회 (변화 없으면 자동 종료)
📜 스크롤 1/15 진행 중...
   📊 현재 이미지 수: 649개 (높이: 11950)
   ✅ 변화 감지됨
📜 스크롤 2/15 진행 중...
   📊 현재 이미지 수: 953개 (높이: 17406)
   ✅ 변화 감지됨
📜 스크롤 3/15 진행 중...
   📊 현재 이미지 수: 1258개 (높이: 22967)
   ✅ 변화 감지됨
📜 스크롤 4/15 진행 중...
   📊 현재 이미지 수: 1258개 (높이: 22967)
   ⚠️ 변화 없음 (1/3)
📜 스크롤 5/15 진행 중...
   📊 현재 이미지 수: 1258개 (높이: 22967)
   ⚠️ 변화 없음 (2/3)
📜 스크롤 6/15 진행 중...
   📊 현재 이미지 수: 1258개 (높이: 22967)
   ⚠️ 변화 없음 (3/3)
   🛑 3번 연속 변화 없음 - 더보기 버튼 확인 중...
   🔘 더보기 버튼 발견!
   ✅ 더보기 버튼 클릭 성공!
📜 스크롤 7/15 진행 중...
   📊 현재 이미지 수: 1279개 (높이: 23463)
   ✅ 변화 감지됨
📜 스크롤 8/15 진행 중...
   📊 현재 이미지 수: 1279개 (높이: 23463)
   ⚠️ 변화 없음 (1/3)
📜 스크롤 9/15 진행 중...
   📊 현재 이미지 수: 1279개 (높이: 23463)
   ⚠️ 변화 없음 (2/3)
📜 스크롤 10/15 진행 중...
   📊 현재 이미지 수: 1279개 (높이: 23463)
   ⚠️ 변화 없음 (3/3)
   🛑 3번 연속 변화 없음 - 더보기 버튼 확인 중...
   🔘 더보기 버튼 발견!
   ✅ 더보기 버튼 클릭 성공!
📜 스크

✅ HTTP 다운로드 성공: 8.4KB
✅ 이미지 17 다운로드 성공!
✅ 이미지 17 저장 완료 (8.4 KB) - 관련성 ✓
📊 진행률: 4.2% (성공: 7, 실패: 0, 필터됨: 10)
📸 이미지 18/405 처리 중...
🔍 이미지 18 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTJZ5dXsHeZ965VMq4a-bGC0e3C...
  ✅ 이미지 18 정보 추출 완료
📸 이미지 18 다운로드 시작
🔗 기본 다운로드 시도: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTJZ5dXsHeZ965VMq4a-bGC0e3C...
✅ HTTP 다운로드 성공: 11.5KB
✅ 이미지 18 다운로드 성공!
✅ 이미지 18 저장 완료 (11.5 KB) - 관련성 ✓
📊 진행률: 4.4% (성공: 8, 실패: 0, 필터됨: 10)
📸 이미지 19/405 처리 중...
🔍 이미지 19 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQC1MMkDKfjpnZpSAiOHNcGO8ce...
  ✅ 이미지 19 정보 추출 완료
📸 이미지 19 다운로드 시작
🔗 기본 다운로드 시도: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQC1MMkDKfjpnZpSAiOHNcGO8ce...
✅ HTTP 다운로드 성공: 6.1KB
✅ 이미지 19 다운로드 성공!
❌ 제외됨 (키워드: 짬뽕): 짜장면 vs. 짬뽕… 건강에 덜 나쁜 것은? - 헬스조...
🗑️ 관련성 없는 이미지 삭제: 19
📊 진행률: 4.7% (성공: 8, 실패: 0, 필터됨: 11)
📸 이미지 20/405 처리 중...
🔍 이미지 20 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstat

✅ HTTP 다운로드 성공: 13.4KB
✅ 이미지 38 다운로드 성공!
✅ 이미지 38 저장 완료 (13.4 KB) - 관련성 ✓
📊 진행률: 9.4% (성공: 21, 실패: 0, 필터됨: 17)
📸 이미지 39/405 처리 중...
🔍 이미지 39 정보 추출 중...
  ✅ 기본 URL 발견: data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw...
  ✅ 이미지 39 정보 추출 완료
📸 이미지 39 다운로드 시작
🔗 기본 다운로드 시도: data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw...
✅ Base64 이미지 저장 성공
✅ 이미지 39 다운로드 성공!
❌ 제외됨 (필수 키워드 없음): 고메 중화짜장 760g | 상품상세 | cj더마켓 - ...
🗑️ 관련성 없는 이미지 삭제: 39
📊 진행률: 9.6% (성공: 21, 실패: 0, 필터됨: 18)
📸 이미지 40/405 처리 중...
🔍 이미지 40 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS0GEP-q3vq1OOEert26Q7o7iaT...
  ✅ 이미지 40 정보 추출 완료
📸 이미지 40 다운로드 시작
🔗 기본 다운로드 시도: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS0GEP-q3vq1OOEert26Q7o7iaT...
✅ HTTP 다운로드 성공: 15.6KB
✅ 이미지 40 다운로드 성공!
❌ 제외됨 (키워드: 맛집): 하노이 짜장면 맛집 - delivery k...
🗑️ 관련성 없는 이미지 삭제: 40
📊 진행률: 9.9% (성공: 21, 실패: 0, 필터됨: 19)
📸 이미지 41/405 처리 중...
🔍 이미지 41 정보 추출 중...
  ✅ 기본 

  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcThJJ8HgV0yLyRIEyDEUOhAfG5c...
  ✅ 이미지 59 정보 추출 완료
📸 이미지 59 다운로드 시작
🔗 기본 다운로드 시도: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcThJJ8HgV0yLyRIEyDEUOhAfG5c...
✅ HTTP 다운로드 성공: 10.2KB
✅ 이미지 59 다운로드 성공!
✅ 이미지 59 저장 완료 (10.2 KB) - 관련성 ✓
📊 진행률: 14.6% (성공: 28, 실패: 0, 필터됨: 31)
📸 이미지 60/405 처리 중...
🔍 이미지 60 정보 추출 중...
  ✅ 기본 URL 발견: data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw...
  ✅ 이미지 60 정보 추출 완료
📸 이미지 60 다운로드 시작
🔗 기본 다운로드 시도: data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw...
✅ Base64 이미지 저장 성공
✅ 이미지 60 다운로드 성공!
❌ 제외됨 (파일 작음: 0.0KB): 진짜 중국집 레시피로 짜장면 만들기...
🗑️ 관련성 없는 이미지 삭제: 60
📊 진행률: 14.8% (성공: 28, 실패: 0, 필터됨: 32)
📸 이미지 61/405 처리 중...
🔍 이미지 61 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTNtmdks3UVrKzDGswiTIUzJpno...
  ✅ 이미지 61 정보 추출 완료
📸 이미지 61 다운로드 시작
🔗 기본 다운로드 시도: https://encrypted-tbn0.gstatic.com/images?q=tbn

✅ 이미지 79 저장 완료 (9.6 KB) - 관련성 ✓
📊 진행률: 19.5% (성공: 40, 실패: 0, 필터됨: 39)
📸 이미지 80/405 처리 중...
🔍 이미지 80 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQpIy4W2ojF9rkQs71jQX5H5E0I...
  ✅ 이미지 80 정보 추출 완료
📸 이미지 80 다운로드 시작
🔗 기본 다운로드 시도: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQpIy4W2ojF9rkQs71jQX5H5E0I...
✅ HTTP 다운로드 성공: 6.1KB
✅ 이미지 80 다운로드 성공!
✅ 이미지 80 저장 완료 (6.1 KB) - 관련성 ✓
📊 진행률: 19.8% (성공: 41, 실패: 0, 필터됨: 39)
📸 이미지 81/405 처리 중...
🔍 이미지 81 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQfDPG8l8Onz3JhWVvpiCiaiHuH...
  ✅ 이미지 81 정보 추출 완료
📸 이미지 81 다운로드 시작
🔗 기본 다운로드 시도: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQfDPG8l8Onz3JhWVvpiCiaiHuH...
✅ HTTP 다운로드 성공: 10.6KB
✅ 이미지 81 다운로드 성공!
❌ 제외됨 (키워드: 짬뽕): 짜장면 vs 짬뽕 - youtube...
🗑️ 관련성 없는 이미지 삭제: 81
📊 진행률: 20.0% (성공: 41, 실패: 0, 필터됨: 40)
📸 이미지 82/405 처리 중...
🔍 이미지 82 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTtUIL--JLa0dBwdJPC8Y

...

✅ HTTP 다운로드 성공: 11.2KB
✅ 이미지 308 다운로드 성공!
✅ 이미지 308 저장 완료 (11.2 KB) - 관련성 ✓
📊 진행률: 76.0% (성공: 180, 실패: 0, 필터됨: 128)
📸 이미지 309/405 처리 중...
🔍 이미지 309 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSkZKhGZzn7-Rhnll0E40vMrJcc...
  ✅ 이미지 309 정보 추출 완료
📸 이미지 309 다운로드 시작
🔗 기본 다운로드 시도: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSkZKhGZzn7-Rhnll0E40vMrJcc...
✅ HTTP 다운로드 성공: 13.0KB
✅ 이미지 309 다운로드 성공!
✅ 이미지 309 저장 완료 (13.0 KB) - 관련성 ✓
📊 진행률: 76.3% (성공: 181, 실패: 0, 필터됨: 128)
📸 이미지 310/405 처리 중...
🔍 이미지 310 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRz9A7iH_brcATLxOtZaaOgafN9...
  ✅ 이미지 310 정보 추출 완료
📸 이미지 310 다운로드 시작
🔗 기본 다운로드 시도: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRz9A7iH_brcATLxOtZaaOgafN9...
✅ HTTP 다운로드 성공: 8.1KB
✅ 이미지 310 다운로드 성공!
❌ 제외됨 (키워드: 맛집): 강동구 짜장면' 맛집 빅데이터 추천순위 top100 -...
🗑️ 관련성 없는 이미지 삭제: 310
📊 진행률: 76.5% (성공: 181, 실패: 0, 필터됨: 129)
📸 이미지 311/405 처리 중...
🔍 이미지 311 정보 추출 중...
  ✅ 기본 URL 발견

✅ HTTP 다운로드 성공: 10.7KB
✅ 이미지 328 다운로드 성공!
✅ 이미지 328 저장 완료 (10.7 KB) - 관련성 ✓
📊 진행률: 81.0% (성공: 186, 실패: 0, 필터됨: 142)
📸 이미지 329/405 처리 중...
🔍 이미지 329 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTQii4GFq401_bu_0qnmF93e4R7...
  ✅ 이미지 329 정보 추출 완료
📸 이미지 329 다운로드 시작
🔗 기본 다운로드 시도: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTQii4GFq401_bu_0qnmF93e4R7...
✅ HTTP 다운로드 성공: 11.4KB
✅ 이미지 329 다운로드 성공!
✅ 이미지 329 저장 완료 (11.4 KB) - 관련성 ✓
📊 진행률: 81.2% (성공: 187, 실패: 0, 필터됨: 142)
📸 이미지 330/405 처리 중...
🔍 이미지 330 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTYhDWOVDiVy1YinANoVdkate9H...
  ✅ 이미지 330 정보 추출 완료
📸 이미지 330 다운로드 시작
🔗 기본 다운로드 시도: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTYhDWOVDiVy1YinANoVdkate9H...
✅ HTTP 다운로드 성공: 8.3KB
✅ 이미지 330 다운로드 성공!
✅ 이미지 330 저장 완료 (8.3 KB) - 관련성 ✓
📊 진행률: 81.5% (성공: 188, 실패: 0, 필터됨: 142)
📸 이미지 331/405 처리 중...
🔍 이미지 331 정보 추출 중...
  ✅ 기본 URL 발견: data:image/gif;base64,R0lGODlhAQABAIAA

✅ HTTP 다운로드 성공: 10.0KB
✅ 이미지 348 다운로드 성공!
❌ 제외됨 (키워드: 맛집): 안국 짜장면' 맛집 빅데이터 추천순위 top18 - 다...
🗑️ 관련성 없는 이미지 삭제: 348
📊 진행률: 85.9% (성공: 196, 실패: 0, 필터됨: 152)
📸 이미지 349/405 처리 중...
🔍 이미지 349 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRNxkfGArWqUNvvMrePtajGjzh8...
  ✅ 이미지 349 정보 추출 완료
📸 이미지 349 다운로드 시작
🔗 기본 다운로드 시도: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRNxkfGArWqUNvvMrePtajGjzh8...
✅ HTTP 다운로드 성공: 9.0KB
✅ 이미지 349 다운로드 성공!
✅ 이미지 349 저장 완료 (9.0 KB) - 관련성 ✓
📊 진행률: 86.2% (성공: 197, 실패: 0, 필터됨: 152)
📸 이미지 350/405 처리 중...
🔍 이미지 350 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcREY1CBSvVrdvHrTG7wVKikbxB5...
  ✅ 이미지 350 정보 추출 완료
📸 이미지 350 다운로드 시작
🔗 기본 다운로드 시도: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcREY1CBSvVrdvHrTG7wVKikbxB5...
✅ HTTP 다운로드 성공: 10.6KB
✅ 이미지 350 다운로드 성공!
✅ 이미지 350 저장 완료 (10.6 KB) - 관련성 ✓
📊 진행률: 86.4% (성공: 198, 실패: 0, 필터됨: 152)
📸 이미지 351/405 처리 중...
🔍 이미지 351 정보 추출 중...
  ✅ 기본 URL 발견:

  ✅ 이미지 369 정보 추출 완료
📸 이미지 369 다운로드 시작
🔗 기본 다운로드 시도: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQtijE9hug1YkK__zl43coCNhTG...
✅ HTTP 다운로드 성공: 11.0KB
✅ 이미지 369 다운로드 성공!
✅ 이미지 369 저장 완료 (11.0 KB) - 관련성 ✓
📊 진행률: 91.1% (성공: 209, 실패: 0, 필터됨: 160)
📸 이미지 370/405 처리 중...
🔍 이미지 370 정보 추출 중...
  ✅ 기본 URL 발견: data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw...
  ✅ 이미지 370 정보 추출 완료
📸 이미지 370 다운로드 시작
🔗 기본 다운로드 시도: data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw...
✅ Base64 이미지 저장 성공
✅ 이미지 370 다운로드 성공!
❌ 제외됨 (파일 작음: 0.0KB): 짜장면 4500원 이 가격 실화냐…완전 거저 먹는 셈 ...
🗑️ 관련성 없는 이미지 삭제: 370
📊 진행률: 91.4% (성공: 209, 실패: 0, 필터됨: 161)
📸 이미지 371/405 처리 중...
🔍 이미지 371 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTbaNsbIWNDQQ5oTyJUIxpPk93V...
  ✅ 이미지 371 정보 추출 완료
📸 이미지 371 다운로드 시작
🔗 기본 다운로드 시도: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTbaNsbIWNDQQ5oTyJUIxpPk93V...
✅ HTTP 다운로드 성공: 6.8KB
✅ 이미지 371 다

✅ 이미지 389 저장 완료 (8.9 KB) - 관련성 ✓
📊 진행률: 96.0% (성공: 216, 실패: 0, 필터됨: 173)
📸 이미지 390/405 처리 중...
🔍 이미지 390 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS0WYHn_DEm4EkXQVbFNCsTtg8V...
  ✅ 이미지 390 정보 추출 완료
📸 이미지 390 다운로드 시작
🔗 기본 다운로드 시도: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS0WYHn_DEm4EkXQVbFNCsTtg8V...
✅ HTTP 다운로드 성공: 13.4KB
✅ 이미지 390 다운로드 성공!
❌ 제외됨 (키워드: 식당): 10화 3000원 짜장면 노포식당 가격의 진실...
🗑️ 관련성 없는 이미지 삭제: 390
📊 진행률: 96.3% (성공: 216, 실패: 0, 필터됨: 174)
📸 이미지 391/405 처리 중...
🔍 이미지 391 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTEnOv03DTiC1ilA1Ltl8s7UMi7...
  ✅ 이미지 391 정보 추출 완료
📸 이미지 391 다운로드 시작
🔗 기본 다운로드 시도: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTEnOv03DTiC1ilA1Ltl8s7UMi7...
✅ HTTP 다운로드 성공: 11.2KB
✅ 이미지 391 다운로드 성공!
✅ 이미지 391 저장 완료 (11.2 KB) - 관련성 ✓
📊 진행률: 96.5% (성공: 217, 실패: 0, 필터됨: 174)
📸 이미지 392/405 처리 중...
🔍 이미지 392 정보 추출 중...
  ✅ 기본 URL 발견: https://encrypted-tbn0.gstatic.com/images?q=t